In [1]:
import pickle
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from seq2seq import *
import pandas as pd
import numpy as np
import torch
import random
import itertools
from sklearn.preprocessing import StandardScaler

In [2]:
def set_seeds(seed):
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
# Convert a string that simulates a list to a real list
def convert_string_list(element):
    # Delete [] of the string
    element = element[0:len(element)]
    # Create a list that contains each code as e.g. 'A'
    ATC_list = list(element.split('; '))
    for index, code in enumerate(ATC_list):
        # Delete '' of the code
        ATC_list[index] = code[0:len(code)]
    return ATC_list

In [4]:
train_set = pd.read_csv('../../Data/train_set.csv')
test_set = pd.read_csv('../../Data/test_set.csv')
val_set = pd.read_csv('../../Data/val_set.csv')
set_seeds(78)

In [5]:
def multiplicate_rows(df):
    # Duplicate each compound the number of ATC codes associated to it, copying its SMILES in new rows
    new_rows = []
    
    for _, row in df.iterrows():
        atc_codes = row['ATC Codes']
        atc_codes_list = convert_string_list(atc_codes)
        
        if len(atc_codes_list) > 1:
            for code in atc_codes_list:
                if len(code) == 5:
                    new_row = row.copy()
                    new_row['ATC Codes'] = code
                    new_rows.append(new_row)
        else:
            if len(atc_codes_list[0]) == 5:
                new_rows.append(row)
    
    new_set = pd.DataFrame(new_rows)
    new_set = new_set.reset_index(drop=True)

    return new_set

#Extract molecular descriptors from the dataframe.
def extract_descriptors(df):
    descriptors = df.iloc[:, 2:-5].values
    
    # descriptors[pd.isinf(descriptors)] = pd.nan
    descriptors[pd.isna(descriptors)] = np.nanmedian(descriptors)

    return torch.tensor(descriptors, dtype=torch.float32)
    
# Create vocabularies
# Tokenize the data
def source(df):
    source = []
    for compound in df['Neutralized SMILES']:
        # A list containing each SMILES character separated
        source.append(list(compound))
    return source
def target(df):
    target = []
    for codes in df['ATC Codes']:  
        code = convert_string_list(codes) 
        # A list of lists, each one containing each ATC code character separated 
        for c in code:
            list_c = list(c)
            target.append(list_c)
    return target

In [6]:
new_train_set = multiplicate_rows(train_set)
new_val_set = multiplicate_rows(val_set)
new_test_set = multiplicate_rows(test_set)

new_test_set.to_csv("onecodeperdrug_test_set.csv", index = False)
new_val_set.to_csv("onecodeperdrug_val_set.csv", index = False)

train_descriptors = extract_descriptors(new_train_set)
test_descriptors = extract_descriptors(new_test_set)
test_descriptors2 = extract_descriptors(test_set)
val_descriptors = extract_descriptors(new_val_set)
val_descriptors2 = extract_descriptors(val_set)

scaler = StandardScaler()
train_descriptors = torch.tensor(scaler.fit_transform(train_descriptors.numpy()), dtype=torch.float32)
val_descriptors = torch.tensor(scaler.transform(val_descriptors.numpy()), dtype=torch.float32)
test_descriptors = torch.tensor(scaler.transform(test_descriptors.numpy()), dtype=torch.float32)
val_descriptors2 = torch.tensor(scaler.transform(val_descriptors2.numpy()), dtype=torch.float32)
test_descriptors2 = torch.tensor(scaler.transform(test_descriptors2.numpy()), dtype=torch.float32)

source_train = source(new_train_set)
source_test = source(new_test_set)
# Test set without duplicated compounds
source_test2 = source(test_set)
source_val = source(new_val_set)
# Val set without duplicated compounds
source_val2 = source(val_set)

target_train = target(new_train_set)
target_test = target(new_test_set)
target_val = target(new_val_set)

# An Index object represents a mapping from the vocabulary to integers (indices) to feed into the models
source_index = index.Index(source_train)
target_index = index.Index(target_train)

# Create tensors
X_train = source_index.text2tensor(source_train)
y_train = target_index.text2tensor(target_train)
X_val = source_index.text2tensor(source_val)
X_val2 = source_index.text2tensor(source_val2)
y_val = target_index.text2tensor(target_val)     
X_test = source_index.text2tensor(source_test)
X_test2 = source_index.text2tensor(source_test2)
y_test = target_index.text2tensor(target_test)

if torch.cuda.is_available():
    X_train = X_train.to("cuda")
    y_train = y_train.to("cuda")
    train_descriptors = train_descriptors.to("cuda") 
    test_descriptors = test_descriptors.to("cuda")
    test_descriptors2 = test_descriptors2.to("cuda")
    val_descriptors = val_descriptors.to("cuda")
    val_descriptors2 = val_descriptors2.to("cuda")
    X_val = X_val.to("cuda")
    X_val2 = X_val2.to("cuda")
    y_val = y_val.to("cuda")
    X_test= X_test.to("cuda")
    y_test = y_test.to("cuda")
    X_test2 = X_test2.to("cuda")

In [7]:
hyperparameters_grid = { 
    'embedding_dim': [32, 64, 128],
    'feedforward_dim': [64, 128, 256],
    'enc_layers': [2, 3, 4],
    'dec_layers': [2, 3, 4],
    'attention_heads': [2, 4],
    'dropout': [0.0, 0.1, 0.2],
    'weight_decays': [10**-4, 10**-5],
    'learning_rates': [10**-3, 10**-4]
}

In [8]:
# Randomly sample from dictionary
random_params = {k: random.sample(v, 1)[0] for k, v in hyperparameters_grid.items()}
print(random_params['embedding_dim'])

64


In [9]:
def random_search(max_evals):
    tested_params = set()
    df_tests = pd.DataFrame(columns = ['#epochs', 'embedding_dim', 'feedforward_dim', 'enc_layers', 'dec_layers', 'attention_heads', 'dropout', 'weight_decay', 'learning_rate', 'Precisionatn nivel1', 'Precisionatn nivel2', 'Precisionatn nivel3', 'Precisionatn nivel4', 'Precision nivel1', 'Precision nivel2', 'Precision nivel3', 'Precision nivel4', 'Recall nivel1', 'Recall nivel2', 'Recall nivel3', 'Recall nivel4', 'Drugs that have at least one match'], index = list(range(max_evals)))
    sys.stdout = open('log.txt', 'w')
    for i in range(max_evals):
        while True:
            random_params = {k: random.sample(v, 1)[0] for k, v in hyperparameters_grid.items()}
            params_tuple = tuple(random_params.values())
            if params_tuple not in tested_params:
                tested_params.add(params_tuple)
                break   
        model = multimodal_models.MultimodalTransformer(
                 source_index, 
                 target_index,
                 max_sequence_length = 800,
                 embedding_dimension = random_params['embedding_dim'],
                 descriptors_dimension = train_descriptors.shape[1],
                 feedforward_dimension = random_params['feedforward_dim'],
                 encoder_layers = random_params['enc_layers'],
                 decoder_layers = random_params['dec_layers'],
                 attention_heads = random_params['attention_heads'],
                 activation = "relu",
                 dropout = random_params['dropout'])   
        model.to("cuda")
        model.fit(
                X_train,
                train_descriptors,
                y_train,
                X_val, 
                val_descriptors,
                y_val, 
                batch_size = 32, 
                epochs = 500, 
                learning_rate = random_params['learning_rates'], 
                weight_decay = random_params['weight_decays'],
                progress_bar = 0, 
                save_path = None
        ) 
        model.load_state_dict(torch.load("best_multimodalmodel.pth", weights_only=True))
        ep = model.early_stopping.best_epoch
        loss, error_rate = model.evaluate(X_val, val_descriptors, y_val)    
        predictions, log_probabilities = search_algorithms.multimodal_beam_search(
            model, 
            X_val,
            val_descriptors,
            predictions = 6, # max length of the predicted sequence
            beam_width = 3,
            batch_size = 32, 
            progress_bar = 0
        )
        output_beam = [target_index.tensor2text(p) for p in predictions]
        predictions2, log_probabilities2 = search_algorithms.multimodal_beam_search(
            model, 
            X_val2,
            val_descriptors2,
            predictions = 6, # max length of the predicted sequence
            beam_width = 3,
            batch_size = 32, 
            progress_bar = 0
        )
        output_beam2 = [target_index.tensor2text(p) for p in predictions2]
        
        predictions_onecodeperdrug = []
        for preds in output_beam:
            interm = []
            for pred in preds:
                clean_pred = pred.replace('<START>', '').replace('<END>', '')
                if len(clean_pred) == 5:
                    interm.append(clean_pred)
            predictions_onecodeperdrug.append(interm)
                
        predictions = []
        for preds in output_beam2:
            interm = []
            for pred in preds:
                clean_pred = pred.replace('<START>', '').replace('<END>', '')
                if len(clean_pred) == 5:
                    interm.append(clean_pred)
            predictions.append(interm)
                
        precisionatn_1, precisionatn_2, precisionatn_3, precisionatn_4 = defined_metrics.precisionatn(predictions_onecodeperdrug, "onecodeperdrug_val_set.csv", 'ATC Codes')
        precision_1, precision_2, precision_3, precision_4 = defined_metrics.precision(predictions, "../../Data/val_set.csv", 'ATC Codes')
        recall_1, recall_2, recall_3, recall_4, comp = defined_metrics.recall(predictions, "../../Data/val_set.csv", 'ATC Codes')
        df_tests.iloc[i, :] = [f"{ep}", f"{random_params['embedding_dim']}", f"{random_params['feedforward_dim']}", f"{random_params['enc_layers']}", f"{random_params['dec_layers']}", f"{random_params['attention_heads']}", f"{random_params['dropout']}", f"{random_params['weight_decays']}", f"{random_params['learning_rates']}", f"{precisionatn_1}", f"{precisionatn_2}", f"{precisionatn_3}", f"{precisionatn_4}", f"{precision_1}", f"{precision_2}", f"{precision_3}", f"{precision_4}", f"{recall_1}", f"{recall_2}", f"{recall_3}", f"{recall_4}", f"{comp}"]
        df_tests.to_csv("multimodaltransformer_results.csv", index = False)
    sys.stdout = sys.__stdout__
    return df_tests

In [10]:
df_tests = random_search(200)

C:\Users\trini\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\trini\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\trini\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\trini\AppData\Local\Programs\Python\Python312\Lib\si

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.lines import Line2D

# Cargar datos
df_res = pd.read_csv("results_multimodaltransformer.csv")
df_res = df_res[["embedding_dim", "enc_layers", "dec_layers", "attention_heads", "dropout", 
                 "weight_decay", "learning_rate", "Precision nivel1", "Recall nivel1"]]

size_map = {32: 100, 64: 200, 128: 300}
df_res['embedding dimension'] = df_res['embedding_dim'].map(size_map)
# Normalizar tamaño de los puntos
# df_res['embedding dimension'] = (df_res['embedding_dim'] - df_res['embedding_dim'].min()) / (df_res['embedding_dim'].max() - df_res['embedding_dim'].min()) * 100 + 50

df_res['encoder and decoder layers'] = df_res.apply(
    lambda row: f'enc_{int(row["enc_layers"])} - dec_{int(row["dec_layers"])}', axis=1)
df_res['weight_decay and learning_rate'] = df_res.apply(
    lambda row: f'WD_{row["weight_decay"]} - LR_{row["learning_rate"]}', axis=1)

# Mapear 'dropout' a formas
dropout_map = {0.0: 'o', 0.1: '^', 0.2: 's'}
df_res['dropout value'] = df_res['dropout'].map(dropout_map)

df_res['edgewidth'] = df_res['attention_heads'].map({2: 1.0, 4: 2.0})
color_palette = sns.color_palette("Pastel1", n_colors=df_res['encoder and decoder layers'].nunique())

# Mapa de colores para 'weight_decay and learning_rate'
unique_wd_lr = df_res['weight_decay and learning_rate'].unique()
color_palette2 = sns.color_palette("Set1", n_colors=len(unique_wd_lr))
wd_lr_color_map = dict(zip(unique_wd_lr, color_palette2))

# Mapa de colores para 'encoder and decoder layers'
unique_enc_dec = df_res['encoder and decoder layers'].unique()
enc_dec_color_map = dict(zip(unique_enc_dec, color_palette))

plt.figure(figsize=(12, 12))

# Dibujar puntos con Matplotlib
for i, row in df_res.iterrows():
    plt.scatter(
        x=row['Precision nivel1'],
        y=row['Recall nivel1'],
        s=row['embedding dimension'],  # Tamaño de los puntos
        color=enc_dec_color_map[row['encoder and decoder layers']],  # Color del punto
        marker=row['dropout value'],  # Forma del punto
        edgecolor=wd_lr_color_map[row['weight_decay and learning_rate']],  # Borde basado en mapa precalculado
        linewidth=row['edgewidth'],  # Grosor del borde
        alpha=0.7
    )
    
legend_elements = [
    Line2D([0], [0], color='none', label='Dropout'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=10, label='Dropout 0.0'),
    Line2D([0], [0], marker='^', color='w', markerfacecolor='black', markersize=10, label='Dropout 0.1'),
    Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label='Dropout 0.2'),
    Line2D([0], [0], color='none', label='Attention heads'),
    Line2D([0], [0], color='black', linewidth=1.0, label='2 attention heads'),
    Line2D([0], [0], color='black', linewidth=2.0, label='4 attention heads'),
    Line2D([0], [0], color='none', label='Encoder - Decoder layers'),
    *[Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) for label, color in enc_dec_color_map.items()],
    Line2D([0], [0], color='none', label='Weight decay - Learning rate'),
    *[Line2D([0], [0], marker='o', color='w', markeredgecolor=color, markerfacecolor='none', markersize=10, label=label) for label, color in wd_lr_color_map.items()]
]

size_mapping = {
    100: "32",
    200: "64",
    300: "128"
}

size_legend = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=size/18, label=f'{size_mapping.get(int(size))}') 
    for size in df_res['embedding dimension'].unique() 
]

plt.legend(title='Hiperparámetros', handles=[Line2D([0], [0], color='none', label='Embedding Dimension')] + size_legend + legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1))

plt.title("Combinación de hiperparámetros", fontsize=16)
plt.xlabel("Precision nivel 1", fontsize=14)
plt.ylabel("Recall nivel 1", fontsize=14)
plt.tight_layout()
plt.savefig('multimodaltransformer_hyperparams.png', bbox_inches='tight')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'results_multimodaltransformer.csv'

In [12]:
df_tests.sort_values(by = "Precision nivel1")

,#epochs,embedding_dim,feedforward_dim,enc_layers,dec_layers,attention_heads,dropout,weight_decay,learning_rate,Precisionatn nivel1,...,Precisionatn nivel4,Precision nivel1,Precision nivel2,Precision nivel3,Precision nivel4,Recall nivel1,Recall nivel2,Recall nivel3,Recall nivel4,Drugs that have at least one match
190,17,128,64,3,4,4,0.2,0.0001,0.001,41.911764705882355,...,60.83333333333333,0.2842377260981911,0.6798780487804877,0.7195121951219511,0.5309278350515464,0.3938128050531152,0.7262872628726288,0.7748741773132019,0.6460481099656357,"[164, 123, 97, 68]"
0,23,128,64,4,4,4,0.2,0.0001,0.001,50.0,...,63.52201257861635,0.3531438415159343,0.7211538461538461,0.7360248447204969,0.5112781954887218,0.5017800746482918,0.7526041666666667,0.7822241940254362,0.6379699248120301,"[208, 161, 133, 90]"
152,10,128,64,2,3,2,0.2,1e-05,0.001,48.345588235294116,...,57.85714285714286,0.3893195521102497,0.6649029982363316,0.7367149758454107,0.39495798319327735,0.45040195233993674,0.7110523221634333,0.8309178743961352,0.5978991596638656,"[189, 138, 119, 75]"
120,19,128,128,3,3,4,0.1,1e-05,0.001,55.69852941176471,...,76.84729064039408,0.40223944875107653,0.7895927601809956,0.7857142857142856,0.5912698412698413,0.5479902383003158,0.824270990447461,0.8595364071554549,0.7624007936507935,"[221, 189, 168, 133]"
73,9,128,64,3,4,2,0.2,1e-05,0.001,51.10294117647059,...,54.60992907801418,0.41602067183462527,0.6399082568807338,0.7058823529411765,0.42857142857142855,0.5210307206431236,0.6829765545361877,0.7901960784313726,0.5568783068783069,"[218, 153, 126, 74]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,27,128,128,2,3,2,0.1,1e-05,0.0001,68.19852941176471,...,81.53846153846153,0.6037898363479759,0.755868544600939,0.8030942334739802,0.5494672754946724,0.6950617283950619,0.8244816118935837,0.8847197106690778,0.8269406392694064,"[284, 237, 219, 186]"
39,34,64,256,4,2,2,0.1,1e-05,0.0001,66.54411764705883,...,72.03389830508475,0.6106804478897501,0.7458033573141484,0.7254464285714286,0.5032840722495893,0.6729687051392477,0.7809252597921663,0.8798788265306123,0.7348111658456487,"[278, 224, 203, 156]"
86,33,64,64,4,2,4,0.0,0.0001,0.0001,67.64705882352942,...,72.31404958677686,0.6115417743324718,0.749699157641396,0.7310924369747899,0.5219512195121949,0.6807206431237438,0.8366927396710792,0.8249099639855942,0.7398373983739839,"[277, 238, 205, 160]"
23,22,128,256,4,3,2,0.1,0.0001,0.0001,68.93382352941177,...,78.3882783882784,0.6192937123169681,0.7928403755868543,0.7791164658634541,0.5405797101449273,0.6966982486362332,0.858910406885759,0.8967775865366228,0.8028985507246377,"[284, 249, 230, 191]"


In [13]:
(df_tests.sort_values(by = "Precision nivel1")).to_csv("multimodaltransformer_sortedresults.csv", index = False)

In [7]:
import pandas as pd
df_tests = pd.read_csv('multimodaltransformer_results.csv')
df_tests['F1 nivel1'] = 2*((df_tests['Precision nivel1'] * df_tests['Recall nivel1'])/(df_tests['Precision nivel1'] + df_tests['Recall nivel1']))
df_tests.sort_values(by = "F1 nivel1", ascending=False).to_csv("multimodaltransformer_sortedresults2.csv", index = False)